In [3]:
import sys
sys.path.append('..')

In [1]:
import random
from concurrent.futures.thread import ThreadPoolExecutor
from pathlib import Path


In [22]:
path = Path('/data/home/webb/file.csv')
path

PosixPath('/data/home/webb/file.csv')

In [23]:
path.with_suffix('.pkl')

PosixPath('/data/home/webb/file.pkl')

In [24]:
path.with_suffix('.csv')

PosixPath('/data/home/webb/file.csv')

In [28]:
# python
import pandas as pd

s = pd.Series([
    pd.Timedelta(minutes=5),
    pd.Timedelta(minutes=7),
    pd.Timedelta(minutes=6),
    pd.NaT  # example missing value
])
s

0   0 days 00:05:00
1   0 days 00:07:00
2   0 days 00:06:00
3               NaT
dtype: timedelta64[ns]

In [27]:

# Timedelta mean (skipna=True by default)
mean_td = s.mean()
print(mean_td)  # e.g. 0 days 00:06:00

# Numeric mean in seconds (useful for further calculations)
mean_seconds = s.dt.total_seconds().mean()
print(mean_seconds)  # e.g. 360.0

# Convert numeric mean back to Timedelta
mean_td_from_seconds = pd.to_timedelta(mean_seconds, unit='s')
print(mean_td_from_seconds)

0 days 00:06:00
360.0
0 days 00:06:00


In [10]:
from pandas import Timedelta, Series, DataFrame

ds = [
    Timedelta('06:21:02.234'),
    Timedelta('04:00:01.234345'),
]
ds = Series(ds)
ds.apply(lambda x: str(x.to_pytimedelta()))

0    6:21:02.234000
1    4:00:01.234345
dtype: object

In [25]:
import time
from data_cleaning.list_rename_move_edf_data import read_edf_time_info
from config.paths import PATHS, PatientDir

pd = PatientDir(PATHS.for_mayo_dir / 'B52K3P3G')
edf_path = list(pd.edf_dir.iterdir())[0]

st = time.perf_counter()
for _ in range(10000):
    read_edf_time_info(edf_path, is_competition_ptnt=False)
et = time.perf_counter() - st
print(et)

21.317613011226058


In [28]:
from data_cleaning.list_rename_move_edf_data import timezone_from_edf_header
import pyedflib

header = pyedflib.highlevel.read_edf_header(str(edf_path))
tz_location = timezone_from_edf_header(header)
tz_location

'UTC+02:00'

In [1]:
from pandas import DataFrame

df = DataFrame({'a': [10, 11, 12]}, index=[4, 5, 6])
df

,a
4,10
5,11
6,12


In [4]:
df.iloc[0]

a    10
Name: 4, dtype: int64

In [16]:
def show_sig_segs(sig_segs):
    for seg in range(n_segs):
        print(f'seg {seg}:')
        for chn in range(n_chn):
            print(sig_segs[seg, chn, :])

In [17]:
import numpy as np
import string

alphabet = string.ascii_lowercase

# reshaping np array
n_segs = 2
seg_n_samples = 3
n_samples = n_segs * seg_n_samples
n_chn = 2

sigs = np.zeros((n_chn, n_samples), dtype=object)
sigs[0, :] = np.arange(n_samples, dtype=int)
sigs[1, :] = np.array([alphabet[i] for i in range(n_samples)])

sigs

array([[0, 1, 2, 3, 4, 5],
       ['a', 'b', 'c', 'd', 'e', 'f']], dtype=object)

In [18]:
desired_sigs = np.array([[[0, 1, 2],
                          ['a', 'b', 'c']],
                         [[3, 4, 5],
                          ['d', 'e', 'f']]])
show_sig_segs(desired_sigs)

seg 0:
['0' '1' '2']
['a' 'b' 'c']
seg 1:
['3' '4' '5']
['d' 'e' 'f']


In [11]:
# Wrong!
seg_sigs1 = sigs.reshape((n_segs, n_chn, seg_n_samples))
show_sig_segs(seg_sigs1)

seg 0:
[0 1 2]
[3 4 5]
seg 1:
['a' 'b' 'c']
['d' 'e' 'f']


In [22]:
# We want to reshape by keeping the n_chn the same and splitting the last dimension into segments
seg_sigs2 = sigs.reshape((n_segs, n_chn, seg_n_samples), order='F')
show_sig_segs(seg_sigs2)

seg 0:
[1 3 5]
[2 4 6]
seg 1:
['a' 'c' 'e']
['b' 'd' 'f']


In [24]:
sigs[0].reshape((n_segs, seg_n_samples))

array([[1, 2, 3],
       [4, 5, 6]], dtype=object)

In [27]:
# Correct version
segs_sigs3 = np.zeros((n_segs, n_chn, seg_n_samples),
                      dtype=object)
for chn in range(n_chn):
    segs_sigs3[:, chn, :] = sigs[chn].reshape((n_segs, seg_n_samples))
show_sig_segs(segs_sigs3)

# Features should be computed along the last axis

seg 0:
[1 2 3]
['a' 'b' 'c']
seg 1:
[4 5 6]
['d' 'e' 'f']


In [2]:
# Setting a DataFrame with a numpy array
import numpy as np
import pandas as pd

arr = np.arange(0, 4).reshape((2, 2))
arr

array([[0, 1],
       [2, 3]])

In [6]:
df = pd.DataFrame(index=['a', 'b', 'c', 'd'], columns=['c1', 'c2', 'c3', 'c4'])
df.fillna('z', inplace=True)
df

,c1,c2,c3,c4
a,z,z,z,z
b,z,z,z,z
c,z,z,z,z
d,z,z,z,z


In [9]:
# Setting just a certain slice
df.loc[['b', 'c'], ['c2', 'c3']] = arr
df

,c1,c2,c3,c4
a,z,z,z,z
b,z,0,1,z
c,z,2,3,z
d,z,z,z,z


In [27]:
# Add columns
df = pd.DataFrame(index=['a', 'b', 'c', 'd'], columns=['c1'])
df.fillna('z', inplace=True)
df

,c1
a,z
b,z
c,z
d,z


In [17]:
feature_cols = ['f1', 'f2']
df[feature_cols] = np.nan

In [18]:
df

,c1,f1,f2
a,z,NaN,NaN
b,z,NaN,NaN
c,z,NaN,NaN
d,z,NaN,NaN


In [28]:
df.loc[('b', 'c'), feature_cols] = 1
df

,c1,f1,f2
a,z,NaN,NaN
b,z,1.0,1.0
c,z,1.0,1.0
d,z,NaN,NaN


In [31]:
df['f1'].dropna().unique()

array([1.])

In [32]:
df

,c1,f1,f2
a,z,NaN,NaN
b,z,1.0,1.0
c,z,1.0,1.0
d,z,NaN,NaN


In [22]:
from concurrent.futures import as_completed, ProcessPoolExecutor, ThreadPoolExecutor
import time
import random


def square(x):
    time.sleep(random.random())
    return x ** 2


nums = [i for i in range(10)]
with ThreadPoolExecutor() as exe:
    futures = exe.map(square, nums)
    # futures = {exe.submit(square, num): num  for num in nums}

    for fut in as_completed(futures):
        num = futures[fut]
        print(f"{num}: {fut.result()}")


AttributeError: 'int' object has no attribute '_condition'

In [24]:
CHUNK_SIZE = 4
file_names = list("ABCDEFGHIJKLMNOPQRSTUVXYZ")
print(len(file_names))
batches = [file_names[i:i + CHUNK_SIZE] for i in range(0, len(file_names), CHUNK_SIZE)]
batches

25


[['A', 'B', 'C', 'D'],
 ['E', 'F', 'G', 'H'],
 ['I', 'J', 'K', 'L'],
 ['M', 'N', 'O', 'P'],
 ['Q', 'R', 'S', 'T'],
 ['U', 'V', 'X', 'Y'],
 ['Z']]

In [28]:
files_infos = {l: i for i, l in enumerate(file_names)}

In [30]:
list(files_infos)[1:5]

['B', 'C', 'D', 'E']

In [12]:
# Rename corrcoefs to corrcoef
from utils.io import pickle_path, save_dataframe_multiformat
from config.paths import PATHS
import pandas as pd

for pdir in PATHS.patient_dirs():
    print(f"Processing {pdir.name}")
    segs = pd.read_pickle(pickle_path(pdir.segments_table))
    segs.rename(columns={'corrcoefs': 'corrcoef'}, inplace=True)
    save_dataframe_multiformat(segs, pdir.segments_table)

Processing K37N36L4D
Processing G39B4L9E
Processing M39K4B3C
Processing B52K3P3G
Processing F5TW95P3X
Processing D63Q51K2N
Processing P4Hk23M7L
Processing P2
Processing P1
Processing P3


In [10]:
pdir = PATHS.patient_dirs()[0]
print(f"Processing {pdir.name}")
segs = pd.read_pickle(pickle_path(pdir.segments_table))

Processing K37N36L4D


In [11]:
segs.rename(columns={'corrcoefs': 'corrcoef'}, inplace=True)

In [9]:
save_dataframe_multiformat(segs, pdir.segments_table)

In [17]:
import pandas as pd
df = pd.DataFrame({'file': ['a', 'a', None, 'b'], 'exists': [True, True, False, True]})
df

,file,exists
0,a,True
1,a,True
2,None,False
3,b,True


In [18]:
existing = df[df['exists']]
existing.index

Index([0, 1, 3], dtype='int64')

In [23]:
i = 0
for file, group in df.groupby('file'):
    print(i)
    i += 1
    print(file)
    # print(group)
    # print()

0
a
1
b
